In [2]:
!pip install tensorflow keras

In [31]:
import keras
from keras.layers import Input, Dense,Conv2D,MaxPooling2D,Flatten,Concatenate,Lambda
from keras.models import Model
import keras.backend as K

In [80]:
letter_input = Input(shape=( 105, 105 , 1 ))
x = Conv2D(64, (10, 10),activation="relu")(letter_input)
x = MaxPooling2D()(x)
x = Conv2D(128, (7, 7),activation="relu")(x)
x = MaxPooling2D()(x)
x = Conv2D(128, (4, 4),activation="relu")(x)
x = MaxPooling2D()(x)
x = Conv2D(256, (4, 4),activation="relu")(x)
out = Flatten()(x)


vision_model = Model(letter_input, out)

letter1 = Input(shape=( 105, 105 , 1))
letter2 = Input(shape=( 105, 105 , 1))

out_1 = vision_model(letter1)
out_2 = vision_model(letter2)

def Manhattan_distance(A,B):
    return K.sum( K.abs( A-B),axis=1,keepdims=True)


merged_vector = Lambda(lambda x:Manhattan_distance(x[0],x[1]), output_shape=lambda inp_shp:(inp_shp[0][0],1))([out_1,out_2])


out_final = Dense(4096,activation="sigmoid")(merged_vector)
final_output = Dense(1,activation="sigmoid")(out_final)

final_model = keras.models.Model(inputs=[letter1, letter2], outputs=final_output)



In [81]:
final_model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [82]:
final_model.layers[2].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_88 (InputLayer)        (None, 105, 105, 1)       0         
_________________________________________________________________
conv2d_142 (Conv2D)          (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_109 (MaxPoolin (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_143 (Conv2D)          (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_110 (MaxPoolin (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_144 (Conv2D)          (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_111 (MaxPoolin (None, 9, 9, 128)         0         
__________

In [83]:
final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_89 (InputLayer)           (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
input_90 (InputLayer)           (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
model_39 (Model)                (None, 9216)         1194816     input_89[0][0]                   
                                                                 input_90[0][0]                   
__________________________________________________________________________________________________
lambda_16 (Lambda)              (None, 1)            0           model_39[1][0]                   
          